In [4]:
import string
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

# load document
in_filename = 'cleanedespn.txt'
doc = load_doc(in_filename)
print(doc[:200])

#grab subset
#print(len(doc))
#doc = doc[:700000]

# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

# save sequences to file
out_filename = 'espnword.txt'
save_doc(sequences, out_filename)

If the rumors are true and Doc Rivers gets a contract extension soon, I just hope the Celtics tell the truth in the ensuing news conference. Don't feed us lines like "Thanks to Doc, our young players 
['if', 'the', 'rumors', 'are', 'true', 'and', 'doc', 'rivers', 'gets', 'a', 'contract', 'extension', 'soon', 'i', 'just', 'hope', 'the', 'celtics', 'tell', 'the', 'truth', 'in', 'the', 'ensuing', 'news', 'conference', 'dont', 'feed', 'us', 'lines', 'like', 'thanks', 'to', 'doc', 'our', 'young', 'players', 'made', 'major', 'strides', 'this', 'season', 'or', 'we', 'would', 'have', 'made', 'the', 'playoffs', 'if', 'we', 'didnt', 'have', 'so', 'many', 'injuries', 'thats', 'a', 'load', 'of', 'crap', 'maybe', 'those', 'arent', 'lies', 'but', 'they', 'seem', 'like', 'fibs', 'along', 'the', 'lines', 'of', 'i', 'dont', 'think', 'those', 'jeans', 'make', 'you', 'look', 'fat', 'at', 'all', 'theyre', 'just', 'the', 'wrong', 'size', 'and', 'i', 'wasnt', 'checking', 'her', 'out', 'i', 'thought', 'i', '

In [16]:
# load
in_filename = 'espnword.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 50)            629350    
_________________________________________________________________
lstm_5 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 12587)             1271287   
Total params: 2,051,537
Trainable params: 2,051,537
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
117185/117185 [==============================] - 147s 1ms/step - loss: 7.2820 - acc: 0.0551
Epoch 2/100
117185/1171

In [23]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

# load cleaned text sequences
in_filename = 'espnword.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

respect yet the fight scenes are better than ever its the first ever sequel that blatantly unapologetically steals moments from previous movies in the series yet theres something endearing about the way it keeps happening i felt terrible for sly after all this movie is a desperate transparent attempt to stay

relevant mcgwire and sully because he could act after the same logic that resembles the past few weeks devouring the overall success guard gets thrown out for the bucks i mean everythingas we found out to be thrown out stealing seconds to break me and i think the hell was


In [22]:
seed_text = "dog"
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

in the worst idea in the time i dont think hes performing karaorocky theres a precedent here game added to the farm c you can find out of the first time in the field that willie deserves to exonerate a hint of potential the strategy just a little desperate for
